Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [15]:
import numpy as np
import matplotlib.pyplot as plt

In [16]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [17]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [18]:
CONSTRAINTS

array([[301,  73],
       [146, 245],
       [  1,  52]])

## TEST PROBLEMS

In [19]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [20]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [21]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [22]:
import numpy as np
from typing import Dict, Any, List

def create_problem_context(values: np.ndarray, 
                           weights: np.ndarray, 
                           constraints: np.ndarray, 
                           num_knapsacks: int) -> Dict[str, Any]:
    """
    Creates a dictionary holding all static problem data.
    This replaces the __init__ method.
    """
    num_items = len(values)
    num_dims = weights.shape[1]
    possible_assignments = list(range(-1, num_knapsacks))
    
    return {
        "values": values,
        "weights": weights,
        "constraints": constraints,
        "num_items": num_items,
        "num_knapsacks": num_knapsacks,
        "num_dims": num_dims,
        "possible_assignments": possible_assignments,
    }

def is_feasible(solution: np.ndarray, 
                problem_data: Dict[str, Any]) -> bool:
    """
    Checks if a given solution violates any constraints.
    Returns True if the solution is feasible, False otherwise.
    """
    try:
        num_knapsacks = problem_data["num_knapsacks"]
        num_dims = problem_data["num_dims"]
        num_items = problem_data["num_items"]
        weights = problem_data["weights"]
        constraints = problem_data["constraints"]
        
        usage = np.zeros((num_knapsacks, num_dims))

        for i in range(num_items):
            k = solution[i]
            if k != -1:
                usage[k] += weights[i]

        if np.any(usage > constraints):
            return False
        
        return True
        
    except IndexError:
        # Safety catch if solution contains an invalid knapsack index
        return False

def calculate_fitness(solution: np.ndarray, 
                      problem_data: Dict[str, Any]) -> float:
    """
    Calculates the fitness (total value) of a solution.
    A solution that is not feasible gets a fitness of 0.
    """

    if not is_feasible(solution, problem_data):
        return 0  
    
    total_value = 0.0
    num_items = problem_data["num_items"]
    values = problem_data["values"]
    
    for i in range(num_items):
        if solution[i] != -1: 
            total_value += values[i]
            
    return total_value

def tweak_solution(solution: np.ndarray, 
                   problem_data: Dict[str, Any], 
                   rng: np.random.Generator) -> np.ndarray:
    """
    Applies a single, random modification to a solution.
    Richiede un generatore 'rng' perché non è più memorizzato.
    """
    neighbor = solution.copy()
    
    num_items = problem_data["num_items"]
    possible_assignments = problem_data["possible_assignments"]
    
    item_to_tweak = rng.integers(0, num_items)
    current_assignment = solution[item_to_tweak]
    
    other_assignments = [a for a in possible_assignments if a != current_assignment]
    new_assignment = rng.choice(other_assignments)
    
    neighbor[item_to_tweak] = new_assignment
    return neighbor

def get_initial_solution(problem_data: Dict[str, Any]) -> np.ndarray:
    """
    Generates the empty initial solution (all -1s)
    for a guaranteed feasible starting point.
    """
    
    #it could be useful to do a greedy initialization here
    num_items = problem_data["num_items"]
    return np.full(shape=num_items, fill_value=-1, dtype=int)

def solve_hill_climber(problem_data: Dict[str, Any], 
                       rng: np.random.Generator, 
                       max_iterations=1000):
    
    current_solution = get_initial_solution(problem_data)
    current_fitness = calculate_fitness(current_solution, problem_data)
    
    fitness_history = [current_fitness] 
    #you could implement a simulated annealing  for probabilistic acceptance of worse solutions to escape from local maxima
    for _ in range(max_iterations):
        neighbor = tweak_solution(current_solution, problem_data, rng)
        neighbor_fitness = calculate_fitness(neighbor, problem_data)
        
        if neighbor_fitness >= current_fitness:
            current_solution = neighbor
            current_fitness = neighbor_fitness
        #else:
            # if np.random.random() < np.exp((neighbor_fitness - current_fitness) / T):
            #     current_solution = neighbor
            #     current_fitness = neighbor_fitness
            # T *= cooling   #where T is the temperature and cooling is a factor < 1
        fitness_history.append(current_fitness)
        # you could add an early stopping when plateau is reached ---
        # if no_improve > N: #choose N relative to max_iterations
        #     break
    return current_solution, current_fitness, fitness_history

In [23]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [24]:
problem_ctx = create_problem_context(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS)
sol, fit, history = solve_hill_climber(problem_ctx, rng, max_iterations=1000)
    
print(f"Soluzione Finale: {sol}")
print(f"Fitness Finale: {fit}")

Soluzione Finale: [1 1 1 1 0 2 0 0 0 1 1 2 1 0 0 0 2 0 0 0]
Fitness Finale: 1065.0


In [25]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [26]:
problem_ctx = create_problem_context(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS)
sol, fit, history = solve_hill_climber(problem_ctx, rng, max_iterations=5000)
    
print(f"Soluzione Finale: {sol}")
print(f"Fitness Finale: {fit}")

Soluzione Finale: [ 4 -1  9  9  8  5  8  1  9 -1  2  6 -1 -1  3  3  2  1 -1 -1 -1  4  5  0
  9  2  4 -1  0  0  5 -1 -1  1  4  0  1 -1  0  8  2  3  4 -1  3 -1  3  1
  1  9  3  7  5  6  8  9 -1 -1 -1 -1 -1  6  7  4  5  6  4 -1  0 -1  1 -1
 -1  2  5  4  1  9 -1 -1  8 -1  7 -1  8  4  6  5 -1  5  9 -1  2  3  0  5
  3  3  8  7]
Fitness Finale: 36565.0


In [27]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(
    1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS, NUM_DIMENSIONS)
)

In [28]:
problem_ctx = create_problem_context(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS)
sol, fit, history = solve_hill_climber(problem_ctx, rng, max_iterations=5000)
    
print(f"Soluzione Finale: {sol}")
print(f"Fitness Finale: {fit}")

Soluzione Finale: [46 86 68 ... -1 23 -1]
Fitness Finale: 1086746.0
